In [13]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, get_scorer_names, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from xgboost import XGBClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from statistics import mode as md

In [14]:
X_test = pd.read_csv("../tanzanian_water_wells/X_test.csv")
X_train = pd.read_csv("../tanzanian_water_wells/X_train.csv")
y_train = pd.read_csv("../tanzanian_water_wells/y_train.csv")

df = pd.concat([X_train, y_train], axis=1)

In [15]:
desc = {'amount_tsh': 'Total static head (amount water available to waterpoint)',
                    'date_recorded': 'The date the row was entered',
                    'funder': 'Who funded the well',
                    'gps_height': 'Altitude of the well',
                    'installer': 'Organization that installed the well',
                    'longitude': 'GPS coordinate',
                    'latitude': 'GPS coordinate',
                    'wpt_name': 'Name of the waterpoint if there is one',
                    'subvillage': 'Geographic location',
                    'region': 'Geographic location',
                    'region_code': 'Geographic location (coded)',
                    'district_code': 'Geographic location (coded)',
                    'lga': 'Geographic location',
                    'ward': 'Geographic location',
                    'population': 'Population around the well',
                    'public_meeting': 'True/False',
                    'recorded_by': 'Group entering this row of data',
                    'scheme_management': 'Who operates the waterpoint',
                    'scheme_name': 'Who operates the waterpoint',
                    'permit': 'If the waterpoint is permitted',
                    'construction_year': 'Year the waterpoint was constructed',
                    'extraction_type': 'The kind of extraction the waterpoint uses',
                    'extraction_type_group': 'The kind of extraction the waterpoint uses',
                    'extraction_type_class': 'The kind of extraction the waterpoint uses',
                    'management': 'How the waterpoint is managed',
                    'management_group': 'How the waterpoint is managed',
                    'payment': 'What the water costs',
                    'payment_type': 'What the water costs',
                    'water_quality': 'The quality of the water',
                    'quality_group': 'The quality of the water',
                    'quantity': 'The quantity of water',
                    'quantity_group': 'The quantity of water',
                    'source': 'The source of the water',
                    'source_type': 'The source of the water',
                    'source_class': 'The source of the water',
                    'waterpoint_type': 'The kind of waterpoint',
                    'waterpoint_type_group': 'The kind of waterpoint'}

In [16]:
# Eliminating null values

df.funder.fillna("Unknown", inplace=True)
df.installer.fillna("Unknown", inplace=True)
df.scheme_management.fillna("None", inplace=True)
df.permit.fillna('Unknown', inplace=True)
df.scheme_name.fillna('Unknown', inplace=True)
df.subvillage.fillna('Unknown', inplace=True)
df.public_meeting.fillna('Unknown', inplace=True)

# Defining the train and test sets

In [17]:
X = df.copy()

columns = ['amount_tsh', 'gps_height', 'population', 'region', 'lga', 
           'scheme_management', 'permit', 'construction_year',
           'extraction_type_group', 'payment', 'management', 
           'quality_group', 'quantity', 'source', 'waterpoint_type']

X = X[columns]

# X['public_meeting'] = X['public_meeting'].map({True: 'Yes', False: 'No', 'Unknown': 'Unknown'})
X['permit'] = X['permit'].map({True: 'Yes', False: 'No', 'Unknown': 'Unknown'})
X['gps_height'] = X['gps_height'].astype('float64')
# X['district_code'] = X['district_code'].astype('float64')
X['population'] = X['population'].astype('float64')
# X['district_code'] = X['district_code'].astype('object')

X_cat = X.drop(list(X.select_dtypes(['float64']).columns), axis=1)
X_numeric = X[list(X.select_dtypes(['float64']).columns)]

y = df['status_group']

X_cat = pd.get_dummies(X_cat)

X = pd.concat([X_numeric, X_cat], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train),
                index = X_train.index,
                columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),
                index = X_test.index,
                columns = X_test.columns)

X_train.reset_index(inplace=True, drop=True)
y_train = y_train.reset_index(drop=True)

In [18]:
X_train_sample = X_train.sample(n=1000, random_state=42)
y_train_sample = y_train.iloc[list(X_train_sample.index)]

# Decision Tree

In [21]:
dtc = DecisionTreeClassifier(random_state=42)

In [40]:
param_grid = {
    'min_samples_split': [1, 5, 10, 20, 40],
    'min_samples_leaf': [1, 5, 10, 20],  
    'max_features': [5, 10, 50, 100, None], 
    'max_depth': [1, 3, 5, 7, 10, None]
}

gs_dtc = GridSearchCV(dtc, param_grid, cv=3, verbose=10)
gs_dtc.fit(X_train, y_train)

Fitting 3 folds for each of 600 candidates, totalling 1800 fits
[CV 1/3; 1/600] START max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=1
[CV 1/3; 1/600] END max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=1;, score=nan total time=   0.0s
[CV 2/3; 1/600] START max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=1
[CV 2/3; 1/600] END max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=1;, score=nan total time=   0.0s
[CV 3/3; 1/600] START max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=1
[CV 3/3; 1/600] END max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=1;, score=nan total time=   0.0s
[CV 1/3; 2/600] START max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=5
[CV 1/3; 2/600] END max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=5;, score=0.543 total time=   0.0s
[CV 2/3; 2/600] START max_depth=1, max_features=5, min_samples_leaf=1, min_samples_split=5

[CV 2/3; 15/600] END max_depth=1, max_features=5, min_samples_leaf=10, min_samples_split=40;, score=0.543 total time=   0.0s
[CV 3/3; 15/600] START max_depth=1, max_features=5, min_samples_leaf=10, min_samples_split=40
[CV 3/3; 15/600] END max_depth=1, max_features=5, min_samples_leaf=10, min_samples_split=40;, score=0.543 total time=   0.0s
[CV 1/3; 16/600] START max_depth=1, max_features=5, min_samples_leaf=20, min_samples_split=1
[CV 1/3; 16/600] END max_depth=1, max_features=5, min_samples_leaf=20, min_samples_split=1;, score=nan total time=   0.0s
[CV 2/3; 16/600] START max_depth=1, max_features=5, min_samples_leaf=20, min_samples_split=1
[CV 2/3; 16/600] END max_depth=1, max_features=5, min_samples_leaf=20, min_samples_split=1;, score=nan total time=   0.0s
[CV 3/3; 16/600] START max_depth=1, max_features=5, min_samples_leaf=20, min_samples_split=1
[CV 3/3; 16/600] END max_depth=1, max_features=5, min_samples_leaf=20, min_samples_split=1;, score=nan total time=   0.0s
[CV 1/3; 17

[CV 1/3; 28/600] END max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=10;, score=0.557 total time=   0.0s
[CV 2/3; 28/600] START max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=10
[CV 2/3; 28/600] END max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=10;, score=0.543 total time=   0.0s
[CV 3/3; 28/600] START max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=10
[CV 3/3; 28/600] END max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=10;, score=0.544 total time=   0.0s
[CV 1/3; 29/600] START max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=20
[CV 1/3; 29/600] END max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=20;, score=0.596 total time=   0.0s
[CV 2/3; 29/600] START max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=20
[CV 2/3; 29/600] END max_depth=1, max_features=10, min_samples_leaf=5, min_samples_split=20;, score=0.543 total time=   0.0s

[CV 1/3; 42/600] END max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=5;, score=0.543 total time=   0.0s
[CV 2/3; 42/600] START max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=5
[CV 2/3; 42/600] END max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=5;, score=0.543 total time=   0.0s
[CV 3/3; 42/600] START max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=5
[CV 3/3; 42/600] END max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=5;, score=0.543 total time=   0.0s
[CV 1/3; 43/600] START max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=10
[CV 1/3; 43/600] END max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=10;, score=0.596 total time=   0.0s
[CV 2/3; 43/600] START max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=10
[CV 2/3; 43/600] END max_depth=1, max_features=50, min_samples_leaf=1, min_samples_split=10;, score=0.613 total time=   0.0s
[CV 

[CV 1/3; 55/600] END max_depth=1, max_features=50, min_samples_leaf=10, min_samples_split=40;, score=0.640 total time=   0.0s
[CV 2/3; 55/600] START max_depth=1, max_features=50, min_samples_leaf=10, min_samples_split=40
[CV 2/3; 55/600] END max_depth=1, max_features=50, min_samples_leaf=10, min_samples_split=40;, score=0.618 total time=   0.0s
[CV 3/3; 55/600] START max_depth=1, max_features=50, min_samples_leaf=10, min_samples_split=40
[CV 3/3; 55/600] END max_depth=1, max_features=50, min_samples_leaf=10, min_samples_split=40;, score=0.543 total time=   0.0s
[CV 1/3; 56/600] START max_depth=1, max_features=50, min_samples_leaf=20, min_samples_split=1
[CV 1/3; 56/600] END max_depth=1, max_features=50, min_samples_leaf=20, min_samples_split=1;, score=nan total time=   0.0s
[CV 2/3; 56/600] START max_depth=1, max_features=50, min_samples_leaf=20, min_samples_split=1
[CV 2/3; 56/600] END max_depth=1, max_features=50, min_samples_leaf=20, min_samples_split=1;, score=nan total time=   0.0

...

[CV 3/3; 572/600] END max_depth=None, max_features=100, min_samples_leaf=10, min_samples_split=5;, score=0.758 total time=   0.2s
[CV 1/3; 573/600] START max_depth=None, max_features=100, min_samples_leaf=10, min_samples_split=10
[CV 1/3; 573/600] END max_depth=None, max_features=100, min_samples_leaf=10, min_samples_split=10;, score=0.756 total time=   0.2s
[CV 2/3; 573/600] START max_depth=None, max_features=100, min_samples_leaf=10, min_samples_split=10
[CV 2/3; 573/600] END max_depth=None, max_features=100, min_samples_leaf=10, min_samples_split=10;, score=0.757 total time=   0.2s
[CV 3/3; 573/600] START max_depth=None, max_features=100, min_samples_leaf=10, min_samples_split=10
[CV 3/3; 573/600] END max_depth=None, max_features=100, min_samples_leaf=10, min_samples_split=10;, score=0.756 total time=   0.2s
[CV 1/3; 574/600] START max_depth=None, max_features=100, min_samples_leaf=10, min_samples_split=20
[CV 1/3; 574/600] END max_depth=None, max_features=100, min_samples_leaf=10, 

[CV 3/3; 584/600] END max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=20;, score=0.756 total time=   0.4s
[CV 1/3; 585/600] START max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=40
[CV 1/3; 585/600] END max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=40;, score=0.757 total time=   0.4s
[CV 2/3; 585/600] START max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=40
[CV 2/3; 585/600] END max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=40;, score=0.759 total time=   0.4s
[CV 3/3; 585/600] START max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=40
[CV 3/3; 585/600] END max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=40;, score=0.756 total time=   0.4s
[CV 1/3; 586/600] START max_depth=None, max_features=None, min_samples_leaf=5, min_samples_split=1
[CV 1/3; 586/600] END max_depth=None, max_features=None, min_samples_leaf=5, 

[CV 1/3; 597/600] END max_depth=None, max_features=None, min_samples_leaf=20, min_samples_split=5;, score=0.751 total time=   0.4s
[CV 2/3; 597/600] START max_depth=None, max_features=None, min_samples_leaf=20, min_samples_split=5
[CV 2/3; 597/600] END max_depth=None, max_features=None, min_samples_leaf=20, min_samples_split=5;, score=0.760 total time=   0.3s
[CV 3/3; 597/600] START max_depth=None, max_features=None, min_samples_leaf=20, min_samples_split=5
[CV 3/3; 597/600] END max_depth=None, max_features=None, min_samples_leaf=20, min_samples_split=5;, score=0.748 total time=   0.3s
[CV 1/3; 598/600] START max_depth=None, max_features=None, min_samples_leaf=20, min_samples_split=10
[CV 1/3; 598/600] END max_depth=None, max_features=None, min_samples_leaf=20, min_samples_split=10;, score=0.751 total time=   0.4s
[CV 2/3; 598/600] START max_depth=None, max_features=None, min_samples_leaf=20, min_samples_split=10
[CV 2/3; 598/600] END max_depth=None, max_features=None, min_samples_leaf

/Users/angeloturri/opt/anaconda3/envs/learn-env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
360 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/angeloturri/opt/anaconda3/envs/learn-env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/angeloturri/opt/anaconda3/envs/learn-env/lib/python3.11/site-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/Users/angeloturri/opt/anaconda3/envs/learn-env/lib/python3.11/site-packages/sklearn

GridSearchCV(cv=3,
             estimator=DecisionTreeClassifier(max_depth=5, max_features=100,
                                              min_samples_leaf=5,
                                              min_samples_split=10),
             param_grid={'max_depth': [1, 3, 5, 7, 10, None],
                         'max_features': [5, 10, 50, 100, None],
                         'min_samples_leaf': [1, 5, 10, 20],
                         'min_samples_split': [1, 5, 10, 20, 40]},
             verbose=10)

In [41]:
gs_dtc.best_params_

{'max_depth': None,
 'max_features': 50,
 'min_samples_leaf': 5,
 'min_samples_split': 20}

In [42]:
dtc = DecisionTreeClassifier(max_depth=5, max_features=100, min_samples_leaf=5, min_samples_split=10)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, max_features=100, min_samples_leaf=5,
                       min_samples_split=10)

In [43]:
preds = dtc.predict(X_test)

In [44]:
report_columns = ['functional', 'functional needs repair', 
                  'non functional', 'accuracy', 'macro avg', 
                  'weighted avg']

report_rows = ['precision', 'recall', 
               'f1-score', 'support']

report = pd.DataFrame(classification_report(y_test, preds, output_dict=True), columns=report_columns, index=report_rows)
                      
matrix = pd.DataFrame(confusion_matrix(y_test, preds))

In [45]:
report

,functional,functional needs repair,non functional,accuracy,macro avg,weighted avg
precision,0.665686,0.0,0.860437,0.708822,0.508707,0.693149
recall,0.954687,0.0,0.494853,0.708822,0.483180,0.708822
f1-score,0.784414,0.0,0.628337,0.708822,0.470917,0.667977
support,8055.000000,1064.0,5731.000000,0.708822,14850.000000,14850.000000


In [46]:
matrix

,0,1,2
0,7690,0,365
1,969,0,95
2,2893,2,2836
